In [18]:
import numpy as np
from tqdm import tqdm, trange
import sys
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
print (sys.version)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [19]:
conda_dep = CondaDependencies().create(python_version='3.6.9')
conda_dep.add_pip_package("azureml-defaults") #add this first
conda_dep.add_pip_package("torch") 
conda_dep.add_pip_package("torchvision") 
conda_dep.add_pip_package("tqdm") 

env_file = 'environment_internal.yml'
with open(env_file,"w") as f:
    f.write(conda_dep.serialize_to_string())
    print(conda_dep.serialize_to_string())
print("Saved dependency info in", env_file)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.9

- pip:
  - azureml-defaults
  - torch
  - torchvision
  - tqdm
channels:
- conda-forge

Saved dependency info in environment_internal.yml


In [6]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

SDK version: 1.0.85
ml-main
synbiolic-main
eastus
f1be6293-6d1c-4e77-bb06-b2083e9aeb20


In [7]:
from azureml.core import Dataset
datastore = ws.get_default_datastore()
datastore.upload_files(files=['./chembl_1mil_dataset.csv'],
                       target_path='./',
                       overwrite=True)

Uploading an estimated of 1 files
Uploading ./chembl_1mil_dataset.csv
Uploaded ./chembl_1mil_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [8]:
input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mid-compute/code/users/aryanmisra/rdkit_testing_ground/1000smiles.csv')])

In [9]:
model_path = "./generative_model_max"

In [10]:
#registering model
from azureml.core.model import Model

model = Model.register(model_path= model_path,
                       model_name="gen_model_JAK2",
                       tags={'drug': "design", 'patient': "RL"},
                       description="Generate molecules",
                       workspace=ws)

Registering model gen_model_JAK2


In [11]:
print('Name:', model.name)
print('Version:', model.version)

Name: gen_model_JAK2
Version: 12


In [12]:


environment = Environment.from_conda_specification('base_env', './environment_internal.yml')

In [13]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python",
                                  source_directory = './',
                                  entry_script="./score.py",
                                  conda_file="../environment_internal.yml")

In [14]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException

inference_config = InferenceConfig(entry_script='score.py', environment=environment)

In [16]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
# prov_config = AksCompute.provisioning_configuration()

aks_name = 'myaks2'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

Creating......................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [147]:
from azureml.core.compute import AksCompute, ComputeTarget
resource_group = ws.resource_group
cluster_name = "myaks"
attach_config = AksCompute.attach_configuration(resource_group = resource_group,
                                                cluster_name = cluster_name,
                                                cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
aks_target = ComputeTarget.attach(ws, "myaks", attach_config)

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '314', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': '82a6980f-0426-45ef-bf5d-7c85135116f8', 'x-ms-ratelimit-remaining-subscription-writes': '1191', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-request-id': '|7596ca13845aa14e9a5b659f0fd9c568.a5c1da7f6ad4ad47.3cbd43b0_', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'EASTUS:20200319T044231Z:82a6980f-0426-45ef-bf5d-7c85135116f8', 'X-Content-Type-Options': 'nosniff', 'Date': 'Thu, 19 Mar 2020 04:42:30 GMT'}
Content: b'{"error":{"code":"BadArgument","message":"A compute with the same name already exists. Updating property: provisioningState for compute is not supported yet.","innererror":{"clientRequestId":"6

In [17]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model

# aks_target = AksCompute(ws,"myaks")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice1", [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Running......................................................................................................................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 6dba95b5-a509-43d7-81de-07053f8b72e7
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: myservice1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image mlmain2c9169d7.azurecr.io/azureml/azureml_615cf2dc4f668ac915e03e848baf2b1f:latest locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation I

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 6dba95b5-a509-43d7-81de-07053f8b72e7
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: myservice1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image mlmain2c9169d7.azurecr.io/azureml/azureml_615cf2dc4f668ac915e03e848baf2b1f:latest locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message":